In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
li = []
for i in range(9):
    temp = pd.read_csv(f"GSE_{2014 + i}.csv")
    li.append(temp)
    
df = pd.concat(li)
df.head()

,service_date,time_period,stop_id,station_name,route_or_line,gated_entries
0,2014-12-31,(00:00:00),place-welln,Wellington,Orange Line,1.00
1,2014-12-31,(00:30:00),place-tumnl,Tufts Medical Center,Orange Line,1.00
2,2014-12-31,(02:00:00),place-sdmnl,Suffolk Downs,Blue Line,1.00
3,2014-12-31,(03:00:00),place-asmnl,Ashmont,Red Line,1.00
4,2014-12-31,(03:00:00),place-dwnxg,Downtown Crossing,Orange Line,0.51


In [3]:
df.dtypes

service_date      object
time_period       object
stop_id           object
station_name      object
route_or_line     object
gated_entries    float64
dtype: object

In [4]:
df['service_date'] = pd.to_datetime(df['service_date'])
df["day"] = df['service_date'].dt.day
df["month"] = df['service_date'].dt.month
df["year"] = df["service_date"].dt.year
df.head()

,service_date,time_period,stop_id,station_name,route_or_line,gated_entries,day,month,year
0,2014-12-31,(00:00:00),place-welln,Wellington,Orange Line,1.00,31,12,2014
1,2014-12-31,(00:30:00),place-tumnl,Tufts Medical Center,Orange Line,1.00,31,12,2014
2,2014-12-31,(02:00:00),place-sdmnl,Suffolk Downs,Blue Line,1.00,31,12,2014
3,2014-12-31,(03:00:00),place-asmnl,Ashmont,Red Line,1.00,31,12,2014
4,2014-12-31,(03:00:00),place-dwnxg,Downtown Crossing,Orange Line,0.51,31,12,2014


In [5]:
df = df.groupby(by=["day", "month", "year", "time_period", "route_or_line"]).sum().reset_index()

In [6]:
df["time_period"] = pd.Categorical(df["time_period"])
df["time_period"]=df["time_period"].astype('category').cat.codes
df.head()

,day,month,year,time_period,route_or_line,gated_entries
0,1,1,2014,0,Blue Line,113.70
1,1,1,2014,0,Green Line,197.25
2,1,1,2014,0,Orange Line,208.53
3,1,1,2014,0,Red Line,339.12
4,1,1,2014,0,Silver Line,25.40


In [7]:
route_encode = pd.get_dummies(df.iloc[:,4:5], prefix="route")
route_encode.head()

,route_Blue Line,route_Green Line,route_Orange Line,route_Red Line,route_Silver Line
0,1,0,0,0,0
1,0,1,0,0,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,0,0,1


In [8]:
df.drop(["route_or_line"], axis=1, inplace=True)
df = pd.concat([df, route_encode], axis=1)
df.columns = df.columns.str.strip().str.replace(' ', '_')
df.head()

,day,month,year,time_period,gated_entries,route_Blue_Line,route_Green_Line,route_Orange_Line,route_Red_Line,route_Silver_Line
0,1,1,2014,0,113.70,1,0,0,0,0
1,1,1,2014,0,197.25,0,1,0,0,0
2,1,1,2014,0,208.53,0,0,1,0,0
3,1,1,2014,0,339.12,0,0,0,1,0
4,1,1,2014,0,25.40,0,0,0,0,1


In [9]:
df.describe

<bound method NDFrame.describe of         day  month  year  time_period  gated_entries  route_Blue_Line  \
0         1      1  2014            0         113.70                1   
1         1      1  2014            0         197.25                0   
2         1      1  2014            0         208.53                0   
3         1      1  2014            0         339.12                0   
4         1      1  2014            0          25.40                0   
...     ...    ...   ...          ...            ...              ...   
720799   31     12  2021           46          13.10                0   
720800   31     12  2021           46           0.90                0   
720801   31     12  2021           47           1.49                0   
720802   31     12  2021           47           0.51                0   
720803   31     12  2021           47           4.00                0   

        route_Green_Line  route_Orange_Line  route_Red_Line  route_Silver_Line  
0       

In [10]:
# df.to_csv('mbta_clean.csv')

In [11]:
# df.sample(frac=0.1).to_csv('mbta_clean_small.csv')

In [12]:
# df.sample(frac=0.05).to_csv('mbta_clean_micro.csv')

In [13]:
df.to_csv("mbta_full.csv", index=False)
train, test = train_test_split(df, test_size=0.33, random_state=42)
train.to_csv("mbta_train.csv")
test.to_csv("mbta_test.csv")

In [14]:
train, test = train_test_split(df.sample(frac=0.1, random_state=1), test_size=0.33, random_state=42)
train.to_csv("mbta_train_small.csv", index=False)
test.to_csv("mbta_test_small.csv", index=False)